# Simulations 2

In [1]:
#Necessary Libraries
import random
import heapq
import math
import csv
from queue import Queue
from heapq import heappush, heappop

## Ford Fulkerson Algorithm Implementation

## Augmenting Path Variation 1 - Shortest Augmenting Path

#### Shortest Augmenting Path (SAP): 
Run Dijkstra’s algorithm while treating the edge lengths as unit distances other than their actual Euclidean distances. This will be effectively the same as running the BFS algorithm.

##### Implementation:
To update the distance (v.g) and predecessor (v.π) attributes for every node in the graph, the algorithm essentially performs a breadth-first search (BFS), beginning at the source node s. Finding the shortest routes between the source and every other node in the graph is the objective. The source node is used by the algorithm to initialise a priority queue. Nodes that are close to the source are found and added to the queue during the first iteration. In the following iterations, the algorithm searches its neighbouring nodes for the node with the lowest value, extracts it from the queue, and, in the event that v.g > u.g + 1, executes the RELAX operation. Until all pertinent nodes have been processed, this process keeps going.

In [2]:
def dijkstra_sap(graph, capacities, flow, source, sink):
    n = len(graph)
    dist = [float('inf')] * n
    dist[source] = 0
    parent = [-1] * n
    heap = [(0, source)]

    while heap:
        d, u = heappop(heap)

        if u == sink:
            break

        for v in range(n):
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                if dist[u] + 1 < dist[v]:
                    dist[v] = dist[u] + 1
                    parent[v] = u
                    heappush(heap, (dist[v], v))

    return parent, dist[sink] != float('inf')


def ford_fulkerson_sap(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = dijkstra_sap(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    total_edges = len(capacities)
    mpl = (total_path_length / paths) / longest_path_length if paths > 0 else 0
    return max_flow, paths, mean_length, total_edges, mpl

## Augmenting Path Variation 2 - DFS-like

#### DFS-like: 
When processing the neighbors v of the current node u (line 9 of Dijkstra’s algorithm above), if v.d is infinity, decrease the key value for v from infinity to a decreasing counter value in Q. If v.d is not infinity, do not change v’s key value. This has
the effect of performing a DFS-like search.

##### Implementation:
The algorithm is a Dijkstra modification that includes a node exploration strategy akin to Depth-First Search (DFS). The source node and the algorithm start a priority queue. Nodes close to the source are discovered and added to the queue during the first iteration, with a decreasing counter value used to determine their priority. In the following iterations, the algorithm looks through the queue for the node with the lowest counter value, investigates the nodes that are next to it, and updates v.g when v.g = ∞. Until all relevant nodes are processed, this iterative procedure continues.

In [3]:
def dfs_like_augmenting_path(graph, capacities, flow, start, end):
    n = len(graph)
    dist = [float('inf')] * n
    dist[start] = 0
    parent = [-1] * n
    heap = [(0, start)]
    counter = -1

    while heap:
        _, u = heapq.heappop(heap)

        for v in range(n):
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                if dist[v] == float('inf'):
                    dist[v] = counter
                    counter -= 1
                    parent[v] = u
                    heapq.heappush(heap, (dist[v], v))

    return parent, dist[end] != float('inf')

def ford_fulkerson_dfs_like(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = dfs_like_augmenting_path(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges

## Augmenting Path Variation 3 - Maximum Capacity

#### Maximum Capacity (MaxCap): 
Modify Dijkstra’s algorithm to find the augmenting path with the maximum capacity. Note that the capacity of the critical edge (cf (p) of the augmenting path p) will be the value of t.d when the modified algorithm completes.

##### Implementation:
To implement the Ford-Fulkerson algorithm with the Maximum Capacity (MaxCap) strategy for finding augmenting paths, we will modify Dijkstra's algorithm to select paths based on the maximum capacity rather than the shortest distance. This approach focuses on maximizing the flow in each augmenting path by choosing the path with the highest minimum edge capacity.

In [4]:
def maxcap_augmenting_path(graph, capacities, flow, start, end):
    n = len(graph)
    max_capacity = [0] * n
    max_capacity[start] = float('inf')
    parent = [-1] * n
    heap = [(-max_capacity[start], start)]

    while heap:
        _, u = heapq.heappop(heap)

        for v in range(n):
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                path_capacity = min(max_capacity[u], capacities.get((u, v), 0) - flow[u][v])
                if path_capacity > max_capacity[v]:
                    max_capacity[v] = path_capacity
                    parent[v] = u
                    heapq.heappush(heap, (-max_capacity[v], v))

    return parent, max_capacity[end] > 0

def ford_fulkerson_maxcap(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = maxcap_augmenting_path(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges

## Augmenting Path Variation 4 - Random

#### Random: 
Same modifications as for DFS-like except use a random value as the key value.

##### Implementation:
We'll alter the path-finding strategy to use random values as the key for prioritising nodes in the search in order to implement the Ford-Fulkerson algorithm with a Random strategy for finding augmenting paths. This approach, in contrast to more deterministic methods like DFS-like or Dijkstra's algorithm, introduces a degree of randomness in the order in which nodes are explored.

In [5]:
def random_dfs_for_augmenting_path(graph, capacities, flow, start, end):
    n = len(graph)
    visited = [False] * n
    parent = [-1] * n

    def dfs(u):
        visited[u] = True
        if u == end:
            return True
        neighbors = [v for v in range(n) if capacities.get((u, v), 0) - flow[u][v] > 0]
        random.shuffle(neighbors)
        for v in neighbors:
            if not visited[v]:
                parent[v] = u
                if dfs(v):
                    return True
        return False

    path_found = dfs(start)
    return parent, path_found

def ford_fulkerson_random(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = random_dfs_for_augmenting_path(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges

# Source-Sink Graph Generator & BFS to find the Longest Acyclic Path

In this part we will be generating a source-sink graph and further be finding the longest acyclic path that will be used further ahead for the implementation of the Ford Fulkerson algorithm, and all the different versions to generate the augmented path.

#### Parameters & Constraints of the generated graph:
* It must have n nodes.
* The n nodes will each be assigned random (x, y) coordinates between the range [0,1]
* Add (u, v) or (v, u) at random to E for each pair of vertices u and v within distance r, but not both (i.e., if the edge (u, v) is already in the graph, do not add edge (v, u)). In doing so, a directed Euclidean neighbour graph is produced.
* Assign a capacity with an integer value chosen at random from the range [1..upperCap] to each edge in the graph.

#### Generation of the Longest Acyclic Path using BFS (Breadth - First Search):
* From the graph generated, select a random node as the source node 's'.
* Beginning from the source node, apply Breadth - First Search in order to find the Longest Acyclic Path.
* From the Longest Acyclic Path generated, get the sink node 't'.

In [6]:
def generate_sink_source_graph(n, r, upperCap):
    
    V = [{'x': random.uniform(0, 1), 'y': random.uniform(0, 1)} for _ in range(n)]

    E = set()

    for i in range(n):
        for j in range(n):
            if i != j:
                distance = math.sqrt((V[i]['x'] - V[j]['x'])**2 + (V[i]['y'] - V[j]['y'])**2)
                if distance <= r:
                    if random.random() < 0.5:
                        if ((i, j) not in E) and ((j, i) not in E):
                            E.add((i, j))
                    else:
                        if ((j, i) not in E) and ((i, j) not in E):
                            E.add((j, i))

    capacities = {edge: random.randint(1, upperCap) for edge in E}

    source = random.randint(0, n-1)

    def bfs_longest_path(src):
        visited = [False] * n
        queue = Queue()
        queue.put((src, [src]))

        longest_path = []
        while not queue.empty():
            current_node, path = queue.get()
            visited[current_node] = True

            for neighbor in [edge[1] for edge in E if edge[0] == current_node]:
                if not visited[neighbor]:
                    new_path = path + [neighbor]
                    queue.put((neighbor, new_path))
                    if len(new_path) > len(longest_path):
                        longest_path = new_path

        return longest_path

    longest_path = bfs_longest_path(source)
    sink = longest_path[-1] if longest_path else None

    return V, E, capacities, source, sink, longest_path

results = []

values_i = [
    (20, 0.2, 5),
    (40, 0.2, 5),
    (20, 0.4, 5),
    (40, 0.4, 5),
    (200, 0.2, 100),
    (400, 0.2, 100),
    (200, 0.4, 100),
    (400, 0.4, 100),
]

def write_graph_to_csv(edges, capacities, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Source', 'Destination', 'Capacity'])
        for edge in edges:
            writer.writerow([edge[0], edge[1], capacities[edge]])

def read_graph_from_csv(filename):
    edges = set()
    capacities = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            source, destination, capacity = int(row[0]), int(row[1]), int(row[2])
            edges.add((source, destination))
            capacities[(source, destination)] = capacity
    return edges, capacities

def print_results(variation, max_flow, paths, mean_length, mpl, total_edges):
    print(f"  {variation}:")
    print(f"    Number of Augmenting Paths: {paths}")
    print(f"    Average Length of Augmenting Paths: {mean_length}")
    print(f"    Mean Proportional Length (MPL): {mpl}")
    print(f"    Total Number of Edges: {total_edges}\n")

for n, r, upperCap in values_i:
    
    V, E, capacities, source, sink, longest_path = generate_sink_source_graph(n, r, upperCap)
    longest_path_length = len(longest_path)

    filename = f'graph_data_{n}_{r}_{upperCap}.csv'
    write_graph_to_csv(E, capacities, filename)

    E_read, capacities_read = read_graph_from_csv(filename)
    
    print(f"Graph Parameters - n: {n}, r: {r}, upperCap: {upperCap}\n")

    max_flow_sap, paths_sap, mean_length_sap, mpl_sap, total_edges_sap = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
    print_results("Shortest Augmenting Path", max_flow_sap, paths_sap, mean_length_sap, total_edges_sap, mpl_sap)

    max_flow_dfs, paths_dfs, mean_length_dfs, mpl_dfs, total_edges_dfs = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
    print_results("DFS-like", max_flow_dfs, paths_dfs, mean_length_dfs, mpl_dfs, total_edges_dfs)

    max_flow_maxcap, paths_maxcap, mean_length_maxcap, mpl_maxcap, total_edges_maxcap = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
    print_results("MaxCap", max_flow_maxcap, paths_maxcap, mean_length_maxcap, mpl_maxcap, total_edges_maxcap)

    max_flow_random, paths_random, mean_length_random, mpl_random, total_edges_random = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
    print_results("Random", max_flow_random, paths_random, mean_length_random, mpl_random, total_edges_random)

    print("--------------------------------------------------\n")

Graph Parameters - n: 20, r: 0.2, upperCap: 5

  Shortest Augmenting Path:
    Number of Augmenting Paths: 1
    Average Length of Augmenting Paths: 3.0
    Mean Proportional Length (MPL): 0.75
    Total Number of Edges: 18

  DFS-like:
    Number of Augmenting Paths: 1
    Average Length of Augmenting Paths: 3.0
    Mean Proportional Length (MPL): 0.75
    Total Number of Edges: 18

  MaxCap:
    Number of Augmenting Paths: 1
    Average Length of Augmenting Paths: 3.0
    Mean Proportional Length (MPL): 0.75
    Total Number of Edges: 18

  Random:
    Number of Augmenting Paths: 1
    Average Length of Augmenting Paths: 3.0
    Mean Proportional Length (MPL): 0.75
    Total Number of Edges: 18

--------------------------------------------------

Graph Parameters - n: 40, r: 0.2, upperCap: 5

  Shortest Augmenting Path:
    Number of Augmenting Paths: 1
    Average Length of Augmenting Paths: 7.0
    Mean Proportional Length (MPL): 0.875
    Total Number of Edges: 81

  DFS-like:
   